In [1]:
import sys

sys.path.append("../")

In [2]:
import pandas as pd
from pathlib import Path
from engine.cross_validation_splits import get_class_disjoint_splits
from tqdm import tqdm

pd.options.display.max_rows = 999

KFOLDS = 5
SEED = 42
FOLDS_PATH = Path("../../data/audio/folds")
FOLDS_PATH.mkdir(parents=True)

FileExistsError: [Errno 17] File exists: '../../data/audio/folds'

In [14]:
base_meta_info = pd.read_csv("../../data/audio/train_meta.tsv", sep="\t")
test_meta_info = pd.read_csv("../../data/audio/test_meta.tsv", sep="\t")

In [15]:
base_meta_info.shape, test_meta_info.shape

((167196, 3), (41377, 2))

In [16]:
splits = get_class_disjoint_splits(base_meta_info.artistid.tolist(), KFOLDS, SEED)

In [17]:
len(splits[0]["train"]) / len(base_meta_info), len(splits[0]["val"]) / len(base_meta_info)

(0.800533505586258, 0.19946649441374195)

In [18]:
for i, split in enumerate(splits):
    train_idx, val_idx = split["train"], split["val"]
    meta_info = base_meta_info.copy()
    meta_info.loc[train_idx, "stage"] = "train"
    meta_info.loc[val_idx, "stage"] = "test"

    # meta_info.to_csv(FOLDS_PATH / f"train_meta_with_stages_fold{i}.tsv", sep="\t", index=False)

In [19]:
train_art = meta_info.loc[meta_info.stage == "train", "artistid"].unique()
test_art = meta_info.loc[meta_info.stage == "test", "artistid"].unique()

In [32]:
len(set(train_art).union(set(test_art))) / (len(train_art) + len(test_art))

1.0

# Cross Entropy postprocessing

In [4]:
folds_path = "../../experiments/audio/F10_AUDIO_CROSSVAL_FOLDN_crop81_adamw_cosinelr_smclr_transformer_roadmap"
new_weights_path = Path("../../experiments/audio/folds_weights_f10")
new_weights_path.mkdir(parents=True)
kfolds = 10
for i in tqdm(range(kfolds)):
    fp = Path(folds_path.replace("FOLDN", f"FOLD{i}"))
    weights_path = fp / "weights/epoch_30.ckpt"
    (new_weights_path / f"fold{i}.ckpt").write_bytes(weights_path.read_bytes())

100%|██████████| 10/10 [00:01<00:00,  8.31it/s]
